# Tigergraph Bindings: Demo of IT Infra Analysis

Uses bindings built into PyGraphistry for Tigergraph:

* Configure DB connection
* Call dynamic endpoints for user-defined endpoints
* Call interpreted-mode query
* Visualize results

## Import and connect

In [12]:
import graphistry

# !pip install graphistry -q
# graphistry.register(key='...', protocol='https', server='www.acme.com', ...)

In [13]:
g = graphistry.tigergraph(
    protocol='http', server='www.acme.org',
    user='tigergraph', pwd='tigergraph', 
    db='Storage', #optional
    #web_port = 14240, api_port = 9000, verbose=True
)

## Dynamic user-defined GSQL endpoints: Call, analyze, & plot

In [14]:
g2 = g.gsql_endpoint(
    'StorageImpact', {'vertexType': 'Service', 'input': 61921, 'input.type': 'Pool'},
    #{'edges': '@@edgeList', 'nodes': '@@nodeList'}
)

print('# edges:', len(g2._edges))

g2.plot()

# edges: 241


## On-the-fly GSQL interpreted queries: Call, analyze, & plot

In [15]:
g3 = g.gsql("""
  INTERPRET QUERY () FOR GRAPH Storage { 
  
    OrAccum<BOOL> @@stop;
    ListAccum<EDGE> @@edgeList;
    SetAccum<vertex> @@set;
    
    @@set += to_vertex("61921", "Pool");

    Start = @@set;

    while Start.size() > 0 and @@stop == false do

      Start = select t from Start:s-(:e)-:t
      where e.goUpper == TRUE
      accum @@edgeList += e
      having t.type != "Service";
    end;

    print @@edgeList;
  }
    """, 
  #{'edges': '@@edgeList', 'nodes': '@@nodeList'} # can skip by default
)       

print('# edges:', len(g3._edges))

g3.plot()

# edges: 241
